In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import json
import praw

import pprint
import bs4 as bs
import urllib.request
import bz2,shutil

import boto3
import os
import time
import subprocess

In [36]:
# Retrieve download links

links_df = pd.read_csv('comment_files/links_dataframe.csv')
links_df.head(15)

,month,link,downloaded,size_in_bytes,comment_count
0,2005-12,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
1,2006-01,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
2,2006-02,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
3,2006-03,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
4,2006-04,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
5,2006-05,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
6,2006-06,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
7,2006-07,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
8,2006-08,https://files.pushshift.io/reddit/comments/RC_...,False,0,0
9,2006-09,https://files.pushshift.io/reddit/comments/RC_...,False,0,0


In [41]:
with open('../keys/reddit_appid.txt') as f:
    APP_ID = f.read().rstrip()
with open('../keys/reddit_secret.txt') as f:
    APP_SECRET = f.read().rstrip()
reddit = praw.Reddit(client_id=APP_ID,
                     client_secret=APP_SECRET,
                     user_agent='script:my.project:v1.0.0 (by /u/Someone')


# for idx, row in links_df[links_df['downloaded'] == False].iloc[12:13].iterrows():
for idx, row in links_df.iloc[28:50].iterrows():
    start = time.time()
    month = row['month']
    url = row['link']
    print('Reddit comments month: {}'.format(month))

    filename = url[url.rfind('/') + 1:]
    s3_url = 's3://aust-galv-aust-finalcap/' + filename
    fp, filesize = download_file(s3_url, s3_bucket='aust-galv-aust-finalcap')
    mongoinfo = mongo_import(month, fp)

    comment_count = filter_comments(mongoinfo)
#     get_posts(reddit, month)
    
    links_df.iloc[idx, 2] = True
    links_df.iloc[idx, 3] = filesize
    links_df.iloc[idx, 4] = comment_count
    links_df.to_csv('comment_files/links_dataframe.csv', index=False)

    print('Done! Time elapsed: {:1.2f}'.format(time.time() - start))
    print('')

Reddit comments month: 2008-04
Extracting file...
Removing file...
Loading to mongodb...


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


Saving relevant comments...


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


33657 comments saved.
Dropping comments source collection...
Done! Time elapsed: 32.40

Reddit comments month: 2008-05
Extracting file...
Removing file...
Loading to mongodb...


...

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


64031 comments saved.
Dropping comments source collection...
Done! Time elapsed: 147.48

Reddit comments month: 2010-01
Extracting file...
Removing file...
Loading to mongodb...


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


Saving relevant comments...
72100 comments saved.
Dropping comments source collection...
Done! Time elapsed: 166.33



/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


In [46]:
def get_posts(praw_reddit, month):
    """We need not only comments for these subreddits, but post information as well. This function looks
    at all distinct posts ids within a month and pulls the post information from the reddit API using PRAW."""

    client = MongoClient()
    collection_name = 'comments-{}'.format(month)
    db = client['myreddit']
    comments = db[collection_name]
    post_ids = comments.distinct('link_id')
    distinct_posts = len(post_ids)
    print('There are {} distinct posts for the month'.format(distinct_posts))

    collection_name = 'posts-{}'.format(month)
    posts = db[collection_name]

    praw_generator = praw_reddit.info(post_ids)
    print('Getting post data...')
    for submission in praw_generator:
        d = {'link_id': submission.id, 
             'title': submission.title, 
             'score': submission.score, 
             'is_self': submission.is_self, 
             'datetime': submission.created_utc, 
             'sub': submission.subreddit.display_name, 
             'permalink': submission.permalink}
        posts.insert_one(d)
    
    client.close()

In [47]:
with open('../keys/reddit_appid.txt') as f:
    APP_ID = f.read().rstrip()
with open('../keys/reddit_secret.txt') as f:
    APP_SECRET = f.read().rstrip()
reddit = praw.Reddit(client_id=APP_ID,
                     client_secret=APP_SECRET,
                     user_agent='script:my.project:v1.0.0 (by /u/Someone')


# for idx, row in links_df[links_df['downloaded'] == False].iloc[12:13].iterrows():
for idx, row in links_df.iloc[29:50].iterrows():
    start = time.time()
    month = row['month']
    url = row['link']
    print('Reddit comments month: {}'.format(month))
    
    get_posts(reddit, month)

    print('Done! Time elapsed: {:1.2f}'.format(time.time() - start))
    print('')

Reddit comments month: 2008-05
There are 9721 distinct posts for the month
Getting post data...
Done! Time elapsed: 67.80

Reddit comments month: 2008-06
There are 9854 distinct posts for the month
Getting post data...
Done! Time elapsed: 67.34

Reddit comments month: 2008-07
There are 9926 distinct posts for the month
Getting post data...
Done! Time elapsed: 67.63

Reddit comments month: 2008-08
There are 11542 distinct posts for the month
Getting post data...
Done! Time elapsed: 77.76

Reddit comments month: 2008-09
There are 18393 distinct posts for the month
Getting post data...
Done! Time elapsed: 121.15

Reddit comments month: 2008-10
There are 21285 distinct posts for the month
Getting post data...
Done! Time elapsed: 139.52

Reddit comments month: 2008-11
There are 14654 distinct posts for the month
Getting post data...
Done! Time elapsed: 95.81

Reddit comments month: 2008-12
There are 9777 distinct posts for the month
Getting post data...
Done! Time elapsed: 64.40

Reddit com

In [45]:
links_df.iloc[25:55, :]

,month,link,downloaded,size_in_bytes,comment_count
25,2008-01,https://files.pushshift.io/reddit/comments/RC_...,True,263972619,452990
26,2008-02,https://files.pushshift.io/reddit/comments/RC_...,True,256564276,441768
27,2008-03,https://files.pushshift.io/reddit/comments/RC_...,True,267934549,463728
28,2008-04,https://files.pushshift.io/reddit/comments/RC_...,True,272655574,468317
29,2008-05,https://files.pushshift.io/reddit/comments/RC_...,True,310404232,536380
30,2008-06,https://files.pushshift.io/reddit/comments/RC_...,True,336060719,577684
31,2008-07,https://files.pushshift.io/reddit/comments/RC_...,True,346089066,592610
32,2008-08,https://files.pushshift.io/reddit/comments/RC_...,True,346626502,595959
33,2008-09,https://files.pushshift.io/reddit/comments/RC_...,True,396060313,680892
34,2008-10,https://files.pushshift.io/reddit/comments/RC_...,True,456690506,789874


In [28]:
temp_col.find_one()

{'_id': ObjectId('5e542549d70d406b412f6299'),
 'link_id': 't3_22515',
 'stickied': False,
 'subreddit_id': 't5_6',
 'controversiality': 0,
 'body': "Microsoft hates it's own products?\r\nWho knew?",
 'author_flair_css_class': None,
 'created_utc': 1136078623,
 'ups': 2,
 'score': 2,
 'subreddit': 'reddit.com',
 'id': 'c2718',
 'parent_id': 't3_22515',
 'edited': False,
 'author_flair_text': None,
 'author': 'Pichu0102',
 'retrieved_on': 1473821517,
 'distinguished': None,
 'gilded': 0}

In [72]:
posts.find_one()

{'_id': ObjectId('5e5419474d3ce5d53043c5e0'),
 'link_id': 'fnf5q',
 'title': 'UW Milwaukee protests- Not as intense as Madison, but we do what we can.',
 'score': 12,
 'is_self': False,
 'datetime': 1297974010.0,
 'sub': 'politics',
 'permalink': '/r/politics/comments/fnf5q/uw_milwaukee_protests_not_as_intense_as_madison/'}